In [10]:

import pandas as pd
import numpy as np
train_norm = pd.read_csv("../norm_Data/train_norm.csv")
val_norm = pd.read_csv('../norm_Data/val_norm.csv')
test_norm = pd.read_csv('../norm_Data/test_norm.csv')
# test_df.drop('ID',axis=1,inplace=True)

train_x = train_norm.iloc[:,:-2]
val_x = val_norm.iloc[:,:-3]
val_y = val_norm['Class']
test_x = test_norm.iloc[:,:-2]

In [11]:
val_x

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,0.952547,0.783957,0.914212,0.243457,0.554224,0.517464,0.492727,0.718992,0.532898,0.604917,...,0.558103,0.520090,0.450570,0.478478,0.663640,0.356294,0.573992,0.391898,0.482215,0.223187
1,0.974645,0.775557,0.881030,0.348989,0.562413,0.556259,0.486371,0.725362,0.513820,0.632380,...,0.301115,0.511370,0.458394,0.538553,0.664713,0.212185,0.599844,0.413159,0.471286,0.220940
2,0.977755,0.778175,0.891005,0.371124,0.546536,0.501536,0.491325,0.714603,0.526201,0.633225,...,0.406440,0.512813,0.455734,0.508279,0.664297,0.513481,0.613494,0.382760,0.467979,0.221650
3,0.974043,0.768632,0.896288,0.278161,0.550941,0.555347,0.480115,0.726685,0.573315,0.608475,...,0.402129,0.512917,0.454701,0.519820,0.666869,0.267575,0.582683,0.448836,0.472224,0.221139
4,0.907239,0.799676,0.926202,0.144041,0.551715,0.505926,0.509252,0.692229,0.732084,0.766447,...,0.414466,0.561623,0.438452,0.504710,0.663002,0.487852,0.585558,0.465442,0.458561,0.195836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28457,0.949011,0.787295,0.877161,0.295433,0.554523,0.511096,0.497572,0.721815,0.517675,0.612193,...,0.614996,0.516465,0.460079,0.560371,0.662828,0.416963,0.578938,0.293360,0.484851,0.225269
28458,0.945305,0.778316,0.860941,0.234556,0.559450,0.507199,0.493978,0.723857,0.533145,0.582815,...,0.508990,0.516793,0.455582,0.490181,0.667539,0.498264,0.566624,0.426702,0.463235,0.219774
28459,0.950701,0.785895,0.881109,0.297947,0.555258,0.510298,0.499762,0.703317,0.546312,0.616077,...,0.588435,0.512587,0.472561,0.532917,0.663234,0.406871,0.578480,0.300059,0.494429,0.226566
28460,0.992943,0.770209,0.853047,0.264133,0.547621,0.507883,0.485311,0.716346,0.582092,0.620888,...,0.569496,0.511599,0.450147,0.473639,0.669858,0.359986,0.560191,0.413390,0.466665,0.219549


In [12]:
valid_normal = val_norm[val_norm.Class == 0]
valid_fraud = val_norm[val_norm.Class == 1]
## functions
def ztest(feature):
    mean = valid_normal[feature].mean()
    std = valid_fraud[feature].std()
    zScore = (valid_fraud[feature].mean() - mean) / (std/np.sqrt(sample_size))

    return zScore

sample_size = len(valid_fraud)
significant_features = []
critical_value = 2.58

cols = list(valid_normal.columns)
for i in cols:
  z_vavlue = ztest(i)

  if (abs(z_vavlue) >= critical_value):
    significant_features.append(i)

significant_features

/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


['V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V9',
 'V10',
 'V11',
 'V12',
 'V14',
 'V16',
 'V17',
 'V18',
 'Class']

In [13]:
#functions
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [14]:
from sklearn.ensemble import IsolationForest 
from sklearn.metrics import classification_report,f1_score

val_contamination = val_y.mean()
print(val_contamination)


0.0010540369615627855


In [15]:
## PCA

from sklearn.decomposition import FastICA

count = 1
frame = pd.DataFrame({'n_ica':0,'len':[0],'colnames':['V0'] ,'f1s':[0]})

n_ica = []

for k in range(10,20):
    n_ica.append(k)
    
    ica = FastICA(n_components=i,random_state=0)
    ica.fit(train_x)
    X_train_ica = ica.transform(train_x)
    X_train_ICA = pd.DataFrame(data=X_train_ica, index=train_x.index)

    X_train_ICA_inverse = ica.inverse_transform(X_train_ICA)
    X_train_ICA_inverse = pd.DataFrame(data=X_train_ICA_inverse, index=train_x.index,columns= train_x.columns)


    X_valid_ica = ica.transform(val_x)
    X_valid_ICA = pd.DataFrame(data=X_valid_ica, index=val_x.index)

    X_valid_ICA_inverse = ica.inverse_transform(X_valid_ICA)
    X_valid_ICA_inverse = pd.DataFrame(data=X_valid_ICA_inverse, index=val_x.index, columns= train_x.columns)

    X_test_ica = ica.transform(test_x)
    X_test_ICA = pd.DataFrame(data=X_test_ica, index=test_x.index )

    X_test_ICA_inverse = ica.inverse_transform(X_test_ICA)
    X_test_ICA_inverse = pd.DataFrame(data=X_test_ICA_inverse, index=test_x.index, columns= train_x.columns)

    X_valid_ICA['Class'] = val_y
    X_valid_ICA_inverse['Class'] = val_y
    
    train_data = X_train_ICA_inverse.copy()
    val_df = X_valid_ICA_inverse[:]
    val_x = val_df.drop('Class',axis=1)


    
    all_cols = list(val_x.columns)
    selected_cols = []
    deselected_cols = all_cols

    for col in selected_cols:
        deselected_cols.remove(col)
        

    maxf1s = []

    for j in range(1,17):
        print('count :' ,count,"번")

        
        length = []
        colnames = []
        f1s = []    
        for col in deselected_cols:
            count+=1
            
            temp_cols = selected_cols.copy()
            temp_cols.append(col)
            
            print("selected",selected_cols)
            print("tempcol",temp_cols)
            
            model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
            model.fit(train_x.loc[:,temp_cols])

            pred = model.predict(val_x.loc[:,temp_cols])
            pred = get_pred_label(pred)
            f1score = f1_score(val_y, pred, average='macro')
            print(classification_report(val_y,pred))
            print(f1score)
            
            length.append(len(temp_cols))
            colnames.append(temp_cols)
            f1s.append(f1score)

        temp_df = pd.DataFrame({'n_ica':i,'len':length,'colnames':colnames ,'f1s':f1s})
        frame = pd.concat([frame,temp_df],axis=0)

        for i in range(len(f1s)):
            if(f1s[i] == max(f1s)):
                print('추가')
                temp_index = i
                selected_cols.append(colnames[i][-1])
                deselected_cols.remove(colnames[i][-1])
                break
        
        
        
    


count : 1 번
selected []
tempcol ['V1']


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/metrics/_classificatio

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.00      0.00      0.00        30

    accuracy                           1.00     28462
   macro avg       0.50      0.50      0.50     28462
weighted avg       1.00      1.00      1.00     28462

0.4997363518121419
selected []
tempcol ['V2']


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/metrics/_classificatio

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.00      0.00      0.00        30

    accuracy                           1.00     28462
   macro avg       0.50      0.50      0.50     28462
weighted avg       1.00      1.00      1.00     28462

0.4997363518121419
selected []
tempcol ['V3']


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.77      0.33      0.47        30

    accuracy                           1.00     28462
   macro avg       0.88      0.67      0.73     28462
weighted avg       1.00      1.00      1.00     28462

0.7323559630611931
selected ['V11', 'V16', 'V18', 'V4', 'V12', 'V2', 'V9', 'V26', 'V22', 'V24', 'V13', 'V3', 'V15', 'V19', 'V14']
tempcol ['V11', 'V16', 'V18', 'V4', 'V12', 'V2', 'V9', 'V26', 'V22', 'V24', 'V13', 'V3', 'V15', 'V19', 'V14', 'V27']


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.89      0.27      0.41        30

    accuracy                           1.00     28462
   macro avg       0.94      0.63      0.70     28462
weighted avg       1.00      1.00      1.00     28462

0.7049260428710196
selected ['V11', 'V16', 'V18', 'V4', 'V12', 'V2', 'V9', 'V26', 'V22', 'V24', 'V13', 'V3', 'V15', 'V19', 'V14']
tempcol ['V11', 'V16', 'V18', 'V4', 'V12', 'V2', 'V9', 'V26', 'V22', 'V24', 'V13', 'V3', 'V15', 'V19', 'V14', 'V28']


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.83      0.33      0.48        30

    accuracy                           1.00     28462
   macro avg       0.92      0.67      0.74     28462
weighted avg       1.00      1.00      1.00     28462

0.7379018553027905
추가


In [27]:
frame2 = frame.reset_index(drop=True).sort_values('f1s',ascending=False)
frame2

,n_ica,len,colnames,f1s
2084,3,5,"[V11, V16, V18, V4, V12]",0.839859
2412,3,5,"[V11, V16, V18, V4, V12]",0.839859
444,3,5,"[V11, V16, V18, V4, V12]",0.839859
1100,3,5,"[V11, V16, V18, V4, V12]",0.839859
3068,3,5,"[V11, V16, V18, V4, V12]",0.839859
...,...,...,...,...
1325,14,1,[V13],0.499384
997,13,1,[V13],0.499384
2965,19,1,[V13],0.499384
669,12,1,[V13],0.499384


In [32]:
frame.n_ica.value_counts()

14    478
15    458
1     380
6     340
7     320
10    298
3     240
9     230
18    228
8     140
13     28
17     28
16     28
19     28
12     28
11     28
0       1
Name: n_ica, dtype: int64

In [33]:
478/28

17.071428571428573

In [35]:
for i in range(1,5):
    print(i,'------------')
    for i in range(6,10):
        print(i)
    

1 ------------
6
7
8
9
2 ------------
6
7
8
9
3 ------------
6
7
8
9
4 ------------
6
7
8
9


In [29]:
frame3 = frame2[frame2.n_ica == 15]
len(frame3)

458

In [31]:
frame3.sort_values('f1s',ascending=False)

,n_ica,len,colnames,f1s
413,15,4,"[V11, V16, V18, V4]",0.820588
2709,15,4,"[V11, V16, V18, V4]",0.820588
1397,15,4,"[V11, V16, V18, V4]",0.820588
741,15,4,"[V11, V16, V18, V4]",0.820588
3037,15,4,"[V11, V16, V18, V4]",0.820588
...,...,...,...,...
1662,15,1,[V22],0.499736
1661,15,1,[V21],0.499736
1664,15,1,[V24],0.499446
1655,15,1,[V15],0.499393


In [ ]:
# X_train_ICA.to_csv("../ica_Data/train_ica.csv",index=False)
# X_train_ICA_inverse.to_csv("../ica_Data/train_inica.csv",index=False)
# X_valid_ICA.to_csv("../ica_Data/val_ica.csv",index=False)
# X_valid_ICA_inverse.to_csv("../ica_Data/val_inica.csv",index=False)
# X_test_ICA.to_csv("../ica_Data/test_ica.csv",index=False)
# X_test_ICA_inverse.to_csv("../ica_Data/test_inica.csv",index=False)

